In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import matplotlib.pyplot as plt
import time

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [13]:
with open('best_model.pt', 'rb') as f:
    model = torch.load(f, map_location=device)
model.eval()

SiameseNetwork(
  (cnn1): Sequential(
    (0): ReflectionPad2d((1, 1, 1, 1))
    (1): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1))
    (2): ReLU(inplace=True)
    (3): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout2d(p=0.2, inplace=False)
    (5): ReflectionPad2d((1, 1, 1, 1))
    (6): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU(inplace=True)
    (8): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout2d(p=0.2, inplace=False)
    (10): ReflectionPad2d((1, 1, 1, 1))
    (11): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1))
    (12): ReLU(inplace=True)
    (13): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Dropout2d(p=0.2, inplace=False)
  )
  (fc1): Sequential(
    (0): Linear(in_features=6272, out_features=500, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=500, out_features=200, bias=True)
    (3): ReLU(inplac

In [17]:
cos = nn.CosineSimilarity()

In [4]:
image_locs = pd.read_csv('geo_image_locations.csv')
image_locs.head()

,location,label
0,../geological_similarity\andesite\012L6.jpg,0
1,../geological_similarity\andesite\01ITR.jpg,0
2,../geological_similarity\andesite\01LQQ.jpg,0
3,../geological_similarity\andesite\0230P.jpg,0
4,../geological_similarity\andesite\02741.jpg,0


In [11]:
dummy_img = Image.open(image_locs['location'][0]).convert("RGB")
transform = T.ToTensor()
dummy_img = transform(dummy_img)

In [12]:
print(dummy_img.size())
dummy_img = dummy_img.unsqueeze(0)
print(dummy_img.size())

torch.Size([3, 28, 28])
torch.Size([1, 3, 28, 28])


In [19]:
with torch.no_grad():
    dummy_img = dummy_img.to(device)
    v1, v2 = model(dummy_img, dummy_img)
print(v1)
print(v1.size())
print(cos(v1, v2))

tensor([[  904.2946,   131.8694,    -6.5602,    -1.8539,    40.2483,   551.6593,
           500.0600,   150.3830,  -514.0599,   281.0977,   826.2490,   256.2123,
           219.1050,  -288.6044,   585.0936,  -126.9068,   255.2296,   -40.3754,
           143.6588,   119.0904,  -184.0937,    59.4010,    30.3005,   427.7617,
           122.0748,  -266.1637,   -45.3790,   686.5146,   238.9694,   188.5269,
           322.2386,  -109.8788,    10.4488,   343.5501,   727.5720,   569.9228,
          -346.0932,   407.5990,  -282.3198, -1009.6946,  -316.6801,  -469.9283,
           188.5190,  -639.3193,   190.0857,    89.0589,  -322.3192,  -220.8505,
          -806.3140,  -242.7900]], device='cuda:0')
torch.Size([1, 50])
tensor([1.], device='cuda:0')


In [20]:
temp_embedding = v1.cpu().detach().numpy()
temp_embedding

array([[  904.29456  ,   131.86935  ,    -6.560202 ,    -1.8538573,
           40.24828  ,   551.6593   ,   500.05997  ,   150.38301  ,
         -514.0599   ,   281.09772  ,   826.24896  ,   256.21228  ,
          219.105    ,  -288.60437  ,   585.09357  ,  -126.90682  ,
          255.22958  ,   -40.375412 ,   143.65877  ,   119.090416 ,
         -184.0937   ,    59.40103  ,    30.300495 ,   427.7617   ,
          122.07479  ,  -266.16373  ,   -45.379    ,   686.51465  ,
          238.9694   ,   188.52693  ,   322.23862  ,  -109.87884  ,
           10.448791 ,   343.5501   ,   727.57196  ,   569.9228   ,
         -346.09323  ,   407.59903  ,  -282.3198   , -1009.6946   ,
         -316.68015  ,  -469.92825  ,   188.51903  ,  -639.3193   ,
          190.08571  ,    89.05893  ,  -322.31918  ,  -220.85052  ,
         -806.314    ,  -242.79004  ]], dtype=float32)

In [21]:
embeddings = None
with torch.no_grad():
    start = time.time()
    for i in range(len(image_locs)):
        img = Image.open(image_locs['location'][i]).convert("RGB")
        img = transform(img)
        img = img.unsqueeze(0)  # adds dimension at 0 to represent batch size of 1
        img = img.to(device)
        v1, v2 = model(img, img)
        if embeddings==None:
            embeddings=v1
        else:
            embeddings = torch.cat((embeddings, v1), 0)
    end = time.time()
    print("Completed in: {} seconds".format((end-start)))
    print("Average time: {} seconds per embedding".format((end-start)/len(image_locs)))

Completed in: 56.27270579338074 seconds
Average time: 0.0018758819185739294 seconds per embedding


In [23]:
embeddings.shape

torch.Size([29998, 50])

In [26]:
numpy_embedding = embeddings.cpu().detach().numpy()
numpy_embedding.shape

(29998, 50)

In [27]:
np.save("geo_siamese_embeds.npy", numpy_embedding)